# BackTesting 


In [4]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-I.csv', parse_dates=['Date'], index_col=['Date'])
data = data.loc['2020-01-01':]
data

,Close,ema20,ema50,rsi,macd,macd_signal,Price_Up
Date,,,,,,,
2020-01-01,7200.174316,7290.616103,7572.187445,45.182876,-61.114299,-85.231483,0
2020-01-02,6985.470215,7261.554590,7549.178926,39.271849,-79.166603,-84.018507,1
2020-01-03,7344.884277,7269.490751,7541.167371,50.861068,-63.736756,-79.962157,1
2020-01-04,7410.656738,7282.935130,7536.049307,52.642173,-45.674693,-73.104664,1
2020-01-05,7411.317383,7295.162012,7531.157859,52.660734,-30.950295,-64.673790,1
...,...,...,...,...,...,...,...
2024-12-21,97224.726562,99069.601344,92289.965450,48.952697,1965.864891,3012.484494,0
2024-12-22,95104.937500,98692.014312,92400.356511,44.934433,1437.643624,2697.516320,0
2024-12-23,94686.242188,98310.512205,92489.999087,44.163328,974.011455,2352.815347,1


In [3]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, plot_heatmaps
import talib 

class SMAcross(Strategy):
    def init(self):
        self.ema = self.I(talib.EMA, self.data.Close, timeperiod=20)
                
    def next(self): # Se llama en cada paso o dia.
        price = self.data.Close[-1] # Precio de cierre del ultimo dia, ya que viene el acumulado por cada dia que pasa.

        if price > self.ema[-1] and not self.position.is_long: 
            # self.buy(size=10, tp=price*1.3, sl=price*0.85)
            self.buy(sl=price*0.95)
        elif price <= self.ema[-1] and self.position.is_long:
            self.position.close()

bt = Backtest(data, SMAcross,cash=200000 )
bt.run()

Start                     2020-01-01 00:00:00
End                       2024-12-22 00:00:00
Duration                   1817 days 00:00:00
Exposure Time [%]                   59.735974
Equity Final [$]               1871296.116309
Equity Peak [$]                2035758.335059
Return [%]                         835.648058
Buy & Hold Return [%]             1223.254205
Return (Ann.) [%]                   56.664406
Volatility (Ann.) [%]               69.053085
Sharpe Ratio                         0.820592
Sortino Ratio                        2.152484
Calmar Ratio                         1.110055
Max. Drawdown [%]                    -51.0465
Avg. Drawdown [%]                   -7.418932
Max. Drawdown Duration      860 days 00:00:00
Avg. Drawdown Duration       42 days 00:00:00
# Trades                                  101
Win Rate [%]                        27.722772
Best Trade [%]                      67.130993
Worst Trade [%]                     -5.042587
Avg. Trade [%]                    

In [4]:
bt.plot()

/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest valu

GridPlot(id='p1320', ...)

In [26]:
class RSI(Strategy):
    rsi_period = 22 # atributo de clase
    sl_percent = 0.95
    out_position = 30
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, timeperiod=self.rsi_period) # accedo al atributo de la clase desde una instancia. 
        
    def next(self):
        price = self.data.Close[-1]

        if self.rsi[-1] < self.out_position:
            self.position.close()

        if self.position.is_long:
            return
        
        self.buy(sl= price*self.sl_percent)

bt = Backtest(data, RSI, cash=200000)
starts, heat = bt.optimize(rsi_period=range(5, 30, 1),
            sl_percent=[0.9,0.91,0.92,0.93,],
            maximize='Return [%]',
            return_heatmap=True)

starts


/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


Start                     2020-01-01 00:00:00
End                       2024-12-22 00:00:00
Duration                   1817 days 00:00:00
Exposure Time [%]                   99.394939
Equity Final [$]               4695090.119614
Equity Peak [$]                5122998.869614
Return [%]                         2247.54506
Buy & Hold Return [%]             1223.254205
Return (Ann.) [%]                   88.442243
Volatility (Ann.) [%]              122.062142
Sharpe Ratio                         0.724567
Sortino Ratio                        2.337413
Calmar Ratio                         1.305763
Max. Drawdown [%]                  -67.732218
Avg. Drawdown [%]                   -7.397474
Max. Drawdown Duration      824 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                                   84
Win Rate [%]                        39.285714
Best Trade [%]                     403.256672
Worst Trade [%]                      -7.87189
Avg. Trade [%]                    

In [37]:
starts['_strategy']
plot_heatmaps(heat)


GridPlot(id='p1896', ...)

In [8]:
bt.plot()

/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/Users/ivotonioni/miniconda3/envs/backtest_env/lib/python3.9/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest valu

GridPlot(id='p1705', ...)